In [56]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Polygon, mapping, Point
import folium
from folium.plugins import HeatMap

# === 1. 광진구 행정구역 불러오기 및 필터 ===
shp_path = "Data/emd_20230729 (1)/emd.shp"
gdf_emd = gpd.read_file(shp_path, encoding='euc-kr')

# CRS 설정
if gdf_emd.crs is None:
    gdf_emd.set_crs(epsg=5179, inplace=True)

# 서울시, 광진구 필터링
seoul_gdf = gdf_emd[gdf_emd["EMD_CD"].astype(str).str.startswith("11")]
gwangjin_gdf = seoul_gdf[seoul_gdf["EMD_CD"].astype(str).str.startswith("11215")].copy()
gwangjin_gdf = gwangjin_gdf.to_crs(epsg=5179)

# === 2. 500m 격자 생성 ===
minx, miny, maxx, maxy = gwangjin_gdf.total_bounds
grid_size = 500
grid_list = []

x = minx
while x < maxx:
    y = miny
    while y < maxy:
        cell = Polygon([
            (x, y),
            (x + grid_size, y),
            (x + grid_size, y + grid_size),
            (x, y + grid_size)
        ])
        grid_list.append(cell)
        y += grid_size
    x += grid_size

# 격자 GeoDataFrame 생성 및 clip
grid_gdf = gpd.GeoDataFrame(geometry=grid_list, crs="EPSG:5179")
grid_clipped = gpd.clip(grid_gdf, gwangjin_gdf)

# 중심점 계산
centroid_gdf = gpd.GeoDataFrame(geometry=grid_clipped.geometry.centroid, crs="EPSG:5179")

# 지도 중심 계산
center = gwangjin_gdf.to_crs(epsg=4326).geometry.centroid.unary_union.centroid
map_center = [center.y, center.x]

# === 3. 지도 생성 ===
m = folium.Map(location=map_center, zoom_start=14, tiles="cartodbpositron")

# === 4. 광진구 경계 표시 ===
gwangjin_union = gwangjin_gdf.geometry.unary_union
folium.GeoJson(
    data=mapping(gwangjin_union),
    name="광진구 경계",
    style_function=lambda x: {"color": "black", "weight": 3, "fillOpacity": 0}
).add_to(m)

# === 5. 행정동 경계 ===
for geom in gwangjin_gdf.to_crs(epsg=4326).geometry:
    folium.GeoJson(
        data=mapping(geom),
        style_function=lambda x: {"color": "gray", "weight": 1, "fillOpacity": 0}
    ).add_to(m)

# === 6. 격자 표시 ===
for poly in grid_clipped.to_crs(epsg=4326).geometry:
    folium.GeoJson(
        data=mapping(poly),
        style_function=lambda x: {
            "fillColor": "#00000000",
            "color": "blue",
            "weight": 2,
            "opacity": 0.8,
        }
    ).add_to(m)

# === 7. 중심점 번호 라벨 ===
for idx, point in enumerate(centroid_gdf.to_crs(epsg=4326).geometry):
    folium.map.Marker(
        [point.y, point.x],
        icon=folium.DivIcon(
            icon_size=(150, 36),
            icon_anchor=(7, 7),
            html=f'<div style="font-size: 10pt; color: red;"><b>{idx+1}</b></div>'
        )
    ).add_to(m)

# === 8. 공원 위치 표시 ===
df = pd.read_csv('Data/전국도시공원정보표준데이터.csv', encoding='euc-kr')
pattern = "광진구"
filtered = df[
    df["소재지도로명주소"].astype(str).str.contains(pattern, na=False) |
    df["소재지지번주소"].astype(str).str.contains(pattern, na=False)
].copy()

if "위도" in filtered.columns and "경도" in filtered.columns:
    for _, row in filtered.iterrows():
        try:
            lat = float(row["위도"])
            lon = float(row["경도"])
            name = row.get("공원명", "이름 없음")
            folium.Marker(
                location=[lat, lon],
                popup=name,
                icon=folium.Icon(color="green", icon="tree-conifer", prefix="glyphicon")
            ).add_to(m)
        except:
            continue
else:
    print("❗ 위도/경도 컬럼이 없습니다. 주소 기반 좌표 변환 필요.")

# === 9. HeatMap 표시 ===
df_heat = pd.read_csv("Data/빌라데이타.csv")
df_heat["위도"] = pd.to_numeric(df_heat["위도"], errors="coerce")
df_heat["경도"] = pd.to_numeric(df_heat["경도"], errors="coerce")
df_heat_clean = df_heat.dropna(subset=["위도", "경도"])

heat_data = df_heat_clean[["위도", "경도"]].values.tolist()
heat_data = [[lat, lon] for lat, lon in heat_data if isinstance(lat, float) and isinstance(lon, float)]

heat_layer = HeatMap(
    heat_data,
    radius=5,
    blur=5,
    min_opacity=0.3,
    gradient={
        "0.2": "yellow",
        "0.4": "orange",
        "0.6": "red"
    }
)
heat_layer.add_to(m)

# === 10. 레이어 컨트롤 추가 및 출력 ===
folium.LayerControl().add_to(m)

# === 11. 광진구 어린이집 위치 표시 ===
df_kids = pd.read_csv("Data/광진구_실제운영중_어린이집.csv", encoding="utf-8")

# 위경도 컬럼 확인 및 변환
df_kids["위도"] = pd.to_numeric(df_kids["시설 위도(좌표값)"], errors="coerce")
df_kids["경도"] = pd.to_numeric(df_kids["시설 경도(좌표값)"], errors="coerce")

# NaN 제거
df_kids_clean = df_kids.dropna(subset=["위도", "경도"])

# 지도에 마커로 표시
for _, row in df_kids_clean.iterrows():
    try:
        lat = float(row["위도"])
        lon = float(row["경도"])
        name = row.get("어린이집명", "이름 없음")  # 컬럼명이 다를 경우 수정
        folium.Marker(
            location=[lat, lon],
            popup=f"👶 {name}",
            icon=folium.Icon(color="purple", icon="heart", prefix="fa")  # Font Awesome 아이콘 사용
        ).add_to(m)
    except:
        continue

# 결과 보기 (Jupyter 환경에서)
m

# 저장 옵션 (필요 시)
# m.save("final_gwangjin_map.html")

/var/folders/sy/j9p2p97s1z91p7ftjpl63zdr0000gn/T/ipykernel_37626/374552533.py:47: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  center = gwangjin_gdf.to_crs(epsg=4326).geometry.centroid.unary_union.centroid
/var/folders/sy/j9p2p97s1z91p7ftjpl63zdr0000gn/T/ipykernel_37626/374552533.py:47: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  center = gwangjin_gdf.to_crs(epsg=4326).geometry.centroid.unary_union.centroid
/var/folders/sy/j9p2p97s1z91p7ftjpl63zdr0000gn/T/ipykernel_37626/374552533.py:54: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  gwangjin_union = gwangjin_gdf.geometry.unary_union


In [17]:
grid_count = grid_clipped.shape[0]
print(f"광진구 내부 200m 격자는 총 {grid_count}개입니다.")

광진구 내부 200m 격자는 총 93개입니다.


## 청크별 빌라 개수 시각화

In [57]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Polygon, Point, mapping
import folium

# === 1. 광진구 행정구역 불러오기 ===
shp_path = "Data/emd_20230729 (1)/emd.shp"
gdf_emd = gpd.read_file(shp_path, encoding='euc-kr')

if gdf_emd.crs is None:
    gdf_emd.set_crs(epsg=5179, inplace=True)

# 광진구 필터
gwangjin_gdf = gdf_emd[gdf_emd["EMD_CD"].astype(str).str.startswith("11215")].copy()
gwangjin_gdf = gwangjin_gdf.to_crs(epsg=5179)

# === 2. 500m 격자 생성 ===
minx, miny, maxx, maxy = gwangjin_gdf.total_bounds
grid_size = 500
grid_list = []

x = minx
while x < maxx:
    y = miny
    while y < maxy:
        cell = Polygon([
            (x, y),
            (x + grid_size, y),
            (x + grid_size, y + grid_size),
            (x, y + grid_size)
        ])
        grid_list.append(cell)
        y += grid_size
    x += grid_size

grid_gdf = gpd.GeoDataFrame(geometry=grid_list, crs="EPSG:5179")
grid_clipped = gpd.clip(grid_gdf, gwangjin_gdf)

# === 3. 빌라 좌표 데이터 불러오기 ===
df_villa = pd.read_csv("Data/빌라데이타.csv")
df_villa["위도"] = pd.to_numeric(df_villa["위도"], errors="coerce")
df_villa["경도"] = pd.to_numeric(df_villa["경도"], errors="coerce")
df_villa = df_villa.dropna(subset=["위도", "경도"])

# 빌라 GeoDataFrame으로 변환 (좌표계: EPSG:4326 → EPSG:5179로 변환 필요)
gdf_villa = gpd.GeoDataFrame(
    df_villa,
    geometry=gpd.points_from_xy(df_villa["경도"], df_villa["위도"]),
    crs="EPSG:4326"
).to_crs(epsg=5179)

# === 4. 각 격자별로 빌라 수 세기 ===
grid_clipped["빌라수"] = grid_clipped.geometry.apply(
    lambda poly: gdf_villa.within(poly).sum()
)

# === 5. 지도 중심 계산 ===
center = gwangjin_gdf.to_crs(epsg=4326).geometry.centroid.unary_union.centroid
m = folium.Map(location=[center.y, center.x], zoom_start=14, tiles="cartodbpositron")

# === 6. 격자와 빌라 수 표시 ===
for idx, row in grid_clipped.iterrows():
    geom = row["geometry"]
    count = row["빌라수"]

    # ⭐ Polygon → GeoSeries → 위경도 변환
    geom_wgs = gpd.GeoSeries([geom], crs="EPSG:5179").to_crs(epsg=4326).iloc[0]

    # 격자 표시
    folium.GeoJson(
        data=mapping(geom_wgs),
        style_function=lambda x: {
            "fillColor": "#00000000",
            "color": "blue",
            "weight": 2,
            "opacity": 0.7,
        }
    ).add_to(m)

    # 중심 좌표 계산 (EPSG:5179 → 4326)
    centroid = geom.centroid
    centroid_wgs = gpd.GeoSeries([centroid], crs="EPSG:5179").to_crs(epsg=4326).iloc[0]

    # 빌라 수 텍스트 마커
    folium.map.Marker(
        location=[centroid_wgs.y, centroid_wgs.x],
        icon=folium.DivIcon(
            icon_size=(50, 20),
            icon_anchor=(10, 10),
            html=f'<div style="font-size: 9pt; color: red;"><b>{count}</b></div>'
        )
    ).add_to(m)

# === 7. 광진구 경계선 표시 ===
gwangjin_union = gwangjin_gdf.unary_union
gwangjin_union_wgs = gpd.GeoSeries([gwangjin_union], crs="EPSG:5179").to_crs(epsg=4326).iloc[0]

folium.GeoJson(
    data=mapping(gwangjin_union_wgs),
    name="광진구 경계",
    style_function=lambda x: {"color": "black", "weight": 3, "fillOpacity": 0}
).add_to(m)

# === 8. 개별 행정동 경계 표시 ===
for geom in gwangjin_gdf.geometry:
    geom_wgs = gpd.GeoSeries([geom], crs="EPSG:5179").to_crs(epsg=4326).iloc[0]
    folium.GeoJson(
        data=mapping(geom_wgs),
        style_function=lambda x: {"color": "gray", "weight": 5, "fillOpacity": 0}
    ).add_to(m)

# 결과 보기
m

/var/folders/sy/j9p2p97s1z91p7ftjpl63zdr0000gn/T/ipykernel_37626/2204491476.py:58: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  center = gwangjin_gdf.to_crs(epsg=4326).geometry.centroid.unary_union.centroid
/var/folders/sy/j9p2p97s1z91p7ftjpl63zdr0000gn/T/ipykernel_37626/2204491476.py:58: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  center = gwangjin_gdf.to_crs(epsg=4326).geometry.centroid.unary_union.centroid
/var/folders/sy/j9p2p97s1z91p7ftjpl63zdr0000gn/T/ipykernel_37626/2204491476.py:95: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  gwangjin_union = gwangjin_gdf.unary_union
